In [1]:
pip install transformers

     |████████████████████████████████| 1.3MB 12.7MB/s 
     |████████████████████████████████| 2.9MB 57.0MB/s 
     |████████████████████████████████| 1.1MB 49.5MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=177a34470ee15095f2e968d706bf8f6e98d3b60683e053f5ee0b40e847e21754
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
pip install seqeval

     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=bed7e4bf5300318d532f7a290ad4b11f08cfc2c6b360f4b8275c302798fe172b
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from collections import Counter
import re
import time
import random
import functools

In [4]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
Thu Nov 19 13:20:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                

In [5]:
import re
dump_chars = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~’،ـ؟؛«» '

def clean_word(word):
    word = word.translate(str.maketrans({key: None for key in dump_chars}))
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    word = re.sub(p_tashkeel,"", word)
    
    return word

def load_data(filename):
    f = open(filename, 'r',encoding ='utf-8')
    sents = f.read().split('\n. O\n')
    f.close()
    # tokenize words
    words = [None]*len(sents)
    tokens = [None]*len(sents)
    for i, sent in enumerate(sents):
        sent = sent.split('\n')
        words[i] = []
        tokens[i] = []
        for word in sent:
            line = word.rsplit(' ', 1)
            line[0] = clean_word(line[0])
            if len(line[0]) > 0:
                words[i].append(line[0])
                tokens[i].append(line[1])
                    
                
    return [d for d in words if len(d) > 0], [d for d in tokens if len(d) > 0]

# load data
tag_classes = ['O','B-LOC', 'B-MISC', 'B-ORG', 'B-PERS', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PERS']

In [6]:
from sklearn.model_selection import train_test_split

Asents, Alabels = load_data ('ANERCorp')
Atrain_sents, Atest_sents, Atrain_labels, Atest_labels = train_test_split(Asents, Alabels, test_size=0.1, random_state=2018)
Atrain_sents, Aval_sents, Atrain_labels, Aval_labels = train_test_split(Atrain_sents, Atrain_labels, test_size=0.1, random_state=2018)

In [7]:
Qsents, Qlabels = load_data ('AQMAR')
Qtrain_sents, Qtest_sents, Qtrain_labels, Qtest_labels = train_test_split(Qsents, Qlabels, test_size=0.1, random_state=2018)
Qtrain_sents, Qval_sents, Qtrain_labels, Qval_labels = train_test_split(Qtrain_sents, Qtrain_labels, test_size=0.1, random_state=2018)

In [8]:
with open("ANERCorp_train.tsv", "w", encoding="utf-8") as f:
    for i , row in enumerate (Atrain_sents):
        for word, label in zip(row, Atrain_labels[i]):
            f.write(word + " " + label + "\n")
        f.write(". O\n")
with open("ANERCorp_test.tsv", "w", encoding="utf-8") as f:
    for i , row in enumerate (Atest_sents):
        for word, label in zip(row, Atest_labels[i]):
            f.write(word + " " + label + "\n")
        f.write(". O\n")
with open("ANERCorp_val.tsv", "w", encoding="utf-8") as f:
    for i , row in enumerate (Aval_sents):
        for word, label in zip(row, Aval_labels[i]):
            f.write(word + " " + label + "\n")
        f.write(". O\n")

In [9]:
with open("AQMAR_train.tsv", "w", encoding="utf-8") as f:
    for i , row in enumerate (Qtrain_sents):
        for word, label in zip(row, Qtrain_labels[i]):
            f.write(word + " " + label + "\n")
        f.write(". O\n")
with open("AQMAR_test.tsv", "w", encoding="utf-8") as f:
    for i , row in enumerate (Qtest_sents):
        for word, label in zip(row, Qtest_labels[i]):
            f.write(word + " " + label + "\n")
        f.write(". O\n")
with open("AQMAR_val.tsv", "w", encoding="utf-8") as f:
    for i , row in enumerate (Qval_sents):
        for word, label in zip(row, Qval_labels[i]):
            f.write(word + " " + label + "\n")
        f.write(". O\n")

In [10]:
Atrain_sents, Atrain_labels = load_data ('ANERCorp_train.tsv')
Atest_sents, Atest_labels = load_data ('ANERCorp_test.tsv')
Aval_sents, Aval_labels  = load_data ('ANERCorp_val.tsv')

In [11]:

Qtrain_sents, Qtrain_labels = load_data ('AQMAR_train.tsv')
Qtest_sents, Qtest_labels = load_data ('AQMAR_test.tsv')
Qval_sents, Qval_labels  = load_data ('AQMAR_val.tsv')


In [12]:
from collections import Counter
print(Counter([ label for labels in Atrain_labels for label in labels]))
print(Counter([ label for labels in Atest_labels for label in labels]))
print(Counter([ label for labels in Aval_labels for label in labels]))

Counter({'O': 96263, 'B-LOC': 3616, 'B-PERS': 2869, 'I-PERS': 2275, 'B-ORG': 1617, 'I-ORG': 1134, 'B-MISC': 901, 'I-LOC': 508, 'I-MISC': 418})
Counter({'O': 12361, 'B-LOC': 431, 'B-PERS': 417, 'I-PERS': 308, 'B-ORG': 239, 'I-ORG': 152, 'B-MISC': 106, 'I-MISC': 58, 'I-LOC': 56})
Counter({'O': 10559, 'B-LOC': 383, 'B-PERS': 317, 'I-PERS': 251, 'B-ORG': 170, 'B-MISC': 110, 'I-ORG': 96, 'I-MISC': 62, 'I-LOC': 40})


In [13]:

print(Counter([ label for labels in Qtrain_labels for label in labels]))
print(Counter([ label for labels in Qtest_labels for label in labels]))
print(Counter([ label for labels in Qval_labels for label in labels]))


Counter({'O': 44055, 'B-MISC': 2033, 'B-PERS': 1127, 'I-MISC': 1115, 'B-LOC': 1045, 'I-PERS': 691, 'I-LOC': 449, 'I-ORG': 351, 'B-ORG': 322})
Counter({'O': 5782, 'B-MISC': 256, 'B-PERS': 177, 'I-MISC': 151, 'B-LOC': 149, 'I-PERS': 115, 'I-LOC': 65, 'B-ORG': 45, 'I-ORG': 42})
Counter({'O': 5567, 'B-MISC': 185, 'B-PERS': 150, 'B-LOC': 136, 'I-PERS': 117, 'I-MISC': 94, 'I-LOC': 55, 'B-ORG': 27, 'I-ORG': 27})


In [14]:
#----------------count labels------------------
print ('ANERCorp')
x = Counter([ label for labels in Atrain_labels for label in labels])
y = Counter([ label for labels in Atest_labels for label in labels])
z = Counter([ label for labels in Aval_labels for label in labels])
print ('labels --- train --- test --- val')
print ('other --- ' , x['O'] , '---' ,y['O'], '---' , z['O'])
print ('person --- ' , x['B-PERS']+x['I-PERS'] , '---' ,y['B-PERS']+y['I-PERS'], '---' , z['B-PERS']+z['I-PERS'])
print ('location --- ' , x['B-LOC']+x['I-LOC'] , '---' ,y['B-LOC']+y['I-LOC'], '---' , z['B-LOC']+z['I-LOC'])
print ('organization --- ' , x['B-ORG']+x['I-ORG'] , '---' ,y['B-ORG']+y['I-ORG'], '---' , z['B-ORG']+z['I-ORG'])
print ('MISC --- ' , x['B-MISC']+x['I-MISC'] , '---' ,y['B-MISC']+y['I-MISC'], '---' , z['B-MISC']+z['I-MISC'])

print ('AQMAR')
x = Counter([ label for labels in Qtrain_labels for label in labels])
y = Counter([ label for labels in Qtest_labels for label in labels])
z = Counter([ label for labels in Qval_labels for label in labels])
print ('labels --- train --- test --- val')
print ('other --- ' , x['O'] , '---' ,y['O'], '---' , z['O'])
print ('person --- ' , x['B-PERS']+x['I-PERS'] , '---' ,y['B-PERS']+y['I-PERS'], '---' , z['B-PERS']+z['I-PERS'])
print ('location --- ' , x['B-LOC']+x['I-LOC'] , '---' ,y['B-LOC']+y['I-LOC'], '---' , z['B-LOC']+z['I-LOC'])
print ('organization --- ' , x['B-ORG']+x['I-ORG'] , '---' ,y['B-ORG']+y['I-ORG'], '---' , z['B-ORG']+z['I-ORG'])
print ('MISC --- ' , x['B-MISC']+x['I-MISC'] , '---' ,y['B-MISC']+y['I-MISC'], '---' , z['B-MISC']+z['I-MISC'])

ANERCorp
labels --- train --- test --- val
other ---  96263 --- 12361 --- 10559
person ---  5144 --- 725 --- 568
location ---  4124 --- 487 --- 423
organization ---  2751 --- 391 --- 266
MISC ---  1319 --- 164 --- 172
AQMAR
labels --- train --- test --- val
other ---  44055 --- 5782 --- 5567
person ---  1818 --- 292 --- 267
location ---  1494 --- 214 --- 191
organization ---  673 --- 87 --- 54
MISC ---  3148 --- 407 --- 279


In [15]:
train_sents = Atrain_sents + Qtrain_sents
test_sents = Atest_sents + Qtest_sents
val_sents = Aval_sents + Qval_sents


train_labels = Atrain_labels + Qtrain_labels
test_labels = Atest_labels + Qtest_labels
val_labels = Aval_labels + Qval_labels

In [16]:
x = Counter([ label for labels in train_labels for label in labels])
y = Counter([ label for labels in test_labels for label in labels])
z = Counter([ label for labels in val_labels for label in labels])
print ('labels --- train --- test --- val')
print ('other --- ' , x['O'] , '---' ,y['O'], '---' , z['O'])
print ('person --- ' , x['B-PERS']+x['I-PERS'] , '---' ,y['B-PERS']+y['I-PERS'], '---' , z['B-PERS']+z['I-PERS'])
print ('location --- ' , x['B-LOC']+x['I-LOC'] , '---' ,y['B-LOC']+y['I-LOC'], '---' , z['B-LOC']+z['I-LOC'])
print ('organization --- ' , x['B-ORG']+x['I-ORG'] , '---' ,y['B-ORG']+y['I-ORG'], '---' , z['B-ORG']+z['I-ORG'])
print ('MISC --- ' , x['B-MISC']+x['I-MISC'] , '---' ,y['B-MISC']+y['I-MISC'], '---' , z['B-MISC']+z['I-MISC'])

labels --- train --- test --- val
other ---  140318 --- 18143 --- 16126
person ---  6962 --- 1017 --- 835
location ---  5618 --- 701 --- 614
organization ---  3424 --- 478 --- 320
MISC ---  4467 --- 571 --- 451


In [17]:
print (test_sents[0])
print (test_labels[0])

['ولم', 'يعد', 'من', 'الممكن', 'تبرير', 'سلوك', 'القوى', 'السياسية', 'الشيعية', 'للعرب', 'والمسلمين', 'لا', 'من', 'حيث', 'تحالف', 'هذه', 'القوى', 'مع', 'الاحتلال', 'وتصديها', 'للمقاومة', 'ولا', 'من', 'حيث', 'سياستها', 'وسلوكها', 'الطائفي']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [18]:

print('Max train sentence length:',len(max(train_sents, key=len)))
print('Max val sentence length:',len(max(val_sents, key=len)))
print('Max test sentence length:',len(max(test_sents, key=len)))

Max train sentence length: 176
Max val sentence length: 212
Max test sentence length: 167


In [19]:
def split(sentences,labels, max):
    news = []
    newl = []
    for i , sent in enumerate (sentences):
      tag = labels[i]
      if len (sent) < max:
        news.append(sent)
        newl.append(tag)
      else:
        x = max 
        while (len (sent) > 0):
          if len(sent) < x :
            news.append(sent)
            newl.append(tag)
            sent = []
          else: 
            if tag[x-1]=='O' or tag[x-1]==0 or tag [x] == 'O' or tag [x] == 0:
              news.append(sent[:x])
              newl.append(tag[:x])
              sent = sent[x:]
              tag = tag [x:]
              x = max
            else:
              x = x-1
    return news, newl

In [20]:
train_sents,train_labels = split(train_sents,train_labels, 100)
test_sents,test_labels = split(test_sents,test_labels, 100)
val_sents,val_labels = split(val_sents,val_labels, 100)


print('Max train sentence length:',len(max(train_sents, key=len)))
print('Max val sentence length:',len(max(val_sents, key=len)))
print('Max test sentence length:',len(max(test_sents, key=len)))

Max train sentence length: 100
Max val sentence length: 100
Max test sentence length: 100


In [21]:
bert_model = 'aubmindlab/bert-base-arabertv01'

In [22]:
from transformers import BertTokenizer,AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(bert_model,do_lower_case=False)

Loading BERT tokenizer...


In [23]:
max_len = 256
#------------------------sents processing --------------------------
#-------------------keep all tokens--------------
"""
def encode_sentence(sents,labels):
  tokenized_sents = []
  tokenized_labels = []

  for i , sent in enumerate (sents):
    tokenized_word = []
    tokenized_label = []
    tokenized_word.append("[CLS]")
    tokenized_label.append(0)
    for j , word in enumerate (sent):
      word_tokens = tokenizer.tokenize(word)
      if len(word_tokens) > 0:
        tokenized_word.extend(word_tokens)
        # Use the real label id for the first token of the word, and padding ids for the remaining tokens
        tokenized_label.extend([labels[i][j]] + [0] * (len(word_tokens) - 1))
    tokenized_word.append("[SEP]")
    tokenized_label.append(0)
    special_tokens_count = tokenizer.num_special_tokens_to_add()
    if len(tokenized_word) > max_len - special_tokens_count:
        tokenized_word = tokenized_word[: (max_len - special_tokens_count)]
        tokenized_label = tokenized_label[: (max_len - special_tokens_count)]    
    tokenized_sents.append(tokenized_word)
    tokenized_labels.append(tokenized_label)

  
  return tokenized_sents, tokenized_labels
"""

def encode_sentence(sents,labels):
  tokenized_sents = []
  tokenized_labels = []
  Fasttext_seq = []
  for i , sent in enumerate (sents):
    tokenized_word = []
    tokenized_label = []
    #word_seq = []
    tokenized_word.append("[CLS]")
    tokenized_label.append(0)
    #word_seq.append(0)
    for j , word in enumerate (sent):
      word_tokens = tokenizer.tokenize(word)
      tokenized_word.extend (word_tokens)
      tokenized_label.extend([labels[i][j]] + [0] * (len(word_tokens) - 1))
      """
      if word in embedding.vocab:
        word_seq.extend([embedding.vocab[word].index]+ [0] * (len(word_tokens) - 1))
      else:
        word_seq.extend([embedding.vocab['unk'].index]+ [0] * (len(word_tokens) - 1))
      """
    tokenized_word.append("[SEP]")
    tokenized_label.append(0)
    #word_seq.append(0)
    if len(tokenized_word) > max_len :
        tokenized_word = tokenized_word[: (max_len)]
        tokenized_label = tokenized_label[: (max_len)]
        #word_seq = word_seq [: (max_len)]
    tokenized_sents.append(tokenized_word)
    tokenized_labels.append(tokenized_label)
    #Fasttext_seq.append(word_seq)

  
  return tokenized_sents, tokenized_labels#, Fasttext_seq

"""
      word_tokens = tokenizer.tokenize(word)
      max = 0
      index = 0 
      for k, t in enumerate (word_tokens):
        if (len (t) > max): 
          max = len (t)
          index = k
      tokenized_word.extend (word_tokens)
      for k, t in enumerate (word_tokens):
        if (k == index): 
          tokenized_label.append(labels[i][j])
        else:
          tokenized_label.append(0)
  """

'\n      word_tokens = tokenizer.tokenize(word)\n      max = 0\n      index = 0 \n      for k, t in enumerate (word_tokens):\n        if (len (t) > max): \n          max = len (t)\n          index = k\n      tokenized_word.extend (word_tokens)\n      for k, t in enumerate (word_tokens):\n        if (k == index): \n          tokenized_label.append(labels[i][j])\n        else:\n          tokenized_label.append(0)\n  '

In [24]:
"""
tr_tokenized_sents , tr_tokenized_labels , tr_Fasttext_seq= encode_sentence(train_sents,train_labels)
val_tokenized_sents , val_tokenized_labels, val_Fasttext_seq = encode_sentence(val_sents,val_labels)
te_tokenized_sents , te_tokenized_labels, te_Fasttext_seq = encode_sentence(test_sents,test_labels)
"""
tr_tokenized_sents , tr_tokenized_labels = encode_sentence(train_sents,train_labels)
val_tokenized_sents , val_tokenized_labels = encode_sentence(val_sents,val_labels)
te_tokenized_sents , te_tokenized_labels = encode_sentence(test_sents,test_labels)

In [25]:
print ('tokenization result for sent#1:')
print (train_sents[11])
print (tr_tokenized_sents[11])
#print (tokenized_labels[0])
print (tr_tokenized_labels[11])


#print (tr_Fasttext_seq[11])

tokenization result for sent#1:
['وهو', 'مع', 'فلسطين', 'خميرة', 'عروبة', 'جديدة']
['[CLS]', 'وهو', 'مع', 'فلسطين', 'خم', '##يرة', 'عروبة', 'جديدة', '[SEP]']
[0, 'O', 'O', 'B-LOC', 'O', 0, 'O', 'O', 0]


In [26]:
print('Max tokinized sentence length:',len(max(tr_tokenized_sents, key=len)))
print('Max tokinized sentence length:',len(max(te_tokenized_sents, key=len)))
print('Max tokinized sentence length:',len(max(val_tokenized_sents, key=len)))

Max tokinized sentence length: 177
Max tokinized sentence length: 172
Max tokinized sentence length: 141


In [27]:
max_len = len(max(tr_tokenized_sents, key=len))

In [28]:
#---------------get id, mask, and seg -----------------------
def sents_processing (sents):
  inputs_ids = []
  masks = []
  seg_ids = []
  for sent in sents:
    l = max_len-len(sent)
    inputs_ids.append (tokenizer.convert_tokens_to_ids([word for word in sent])+ [tokenizer.pad_token_id]*l)
    masks.append([1 for word in sent]+ [0]*l)
    seg_ids.append([0 for word in sent ]+ [0]*l)

  return torch.tensor(inputs_ids, dtype=torch.long), torch.tensor(masks, dtype=torch.long), torch.tensor(seg_ids, dtype=torch.long)

In [29]:
train_ids, train_masks, train_seg = sents_processing (tr_tokenized_sents)
val_ids, val_masks, val_seg = sents_processing (val_tokenized_sents)
test_ids, test_masks, test_seg = sents_processing (te_tokenized_sents)

print (train_ids[0])
print (train_masks[0])
print (train_seg[0])

tensor([17028,    12, 25667, 20068, 28764,  8236, 17359,   361, 31529,   883,
         1409,  5095, 21174,   731, 27219, 33293, 57166, 26695, 36960, 32010,
        47486,  3664,    11, 17030, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 

In [30]:
def padding (sents):
  Fasttext_seq = []
  for sent in sents:
    l = max_len-len(sent)
    Fasttext_seq.append(sent+ [0]*l)

  return torch.tensor(Fasttext_seq, dtype=torch.long)


In [31]:
"""
train_Fasttext_seq = padding (tr_Fasttext_seq)
test_Fasttext_seq = padding (te_Fasttext_seq)
val_Fasttext_seq = padding (val_Fasttext_seq)
"""

'\ntrain_Fasttext_seq = padding (tr_Fasttext_seq)\ntest_Fasttext_seq = padding (te_Fasttext_seq)\nval_Fasttext_seq = padding (val_Fasttext_seq)\n'

In [32]:
print('Max inputs_ids length:',len(max(train_ids, key=len)))

Max inputs_ids length: 177


In [33]:

tag2idx = {t: i+1 for i, t in enumerate(tag_classes)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}
def labels_processing (labels):

  labels_ids = []
  for label in labels:
    l = max_len - len (label)
    labels_ids.append ([tag2idx[tag] if tag != 0 else tag for tag in label] + [0]*l)
  return torch.tensor(labels_ids, dtype=torch.long)

In [34]:
train_labels = labels_processing (tr_tokenized_labels)
test_labels = labels_processing (te_tokenized_labels)
val_labels = labels_processing (val_tokenized_labels)

print (train_labels[0])

tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])


In [35]:
print('Max inputs_ids length:',len(max(train_labels, key=len)))

Max inputs_ids length: 177


In [36]:
#-----------------------gazetteer featureas--------------------
"""
per = open('PER_gazetteer.txt','r',encoding='utf-8')
pers_data = per.read().split('\n')
loc = open('LOC_gazetteer.txt','r',encoding='utf-8')
loc_data = per.read().split('\n')
org = open('Org_gazetteer.txt','r',encoding='utf-8')
org_data = per.read().split('\n')

def get_gazetteer_featureas (tokenized_sentence):
  gazetteer_featureas = []
  for sent in tokenized_sentence:
    seq = []
    for word in sent:
      if word in pers_data:
        seq.append (1)
      elif word in loc_data:
        seq.append (2)
      elif word in org_data:
        seq.append (3)
      else:
        seq.append (0) 
    gazetteer_featureas.append(seq)
  return (gazetteer_featureas)   
"""   

"\nper = open('PER_gazetteer.txt','r',encoding='utf-8')\npers_data = per.read().split('\n')\nloc = open('LOC_gazetteer.txt','r',encoding='utf-8')\nloc_data = per.read().split('\n')\norg = open('Org_gazetteer.txt','r',encoding='utf-8')\norg_data = per.read().split('\n')\n\ndef get_gazetteer_featureas (tokenized_sentence):\n  gazetteer_featureas = []\n  for sent in tokenized_sentence:\n    seq = []\n    for word in sent:\n      if word in pers_data:\n        seq.append (1)\n      elif word in loc_data:\n        seq.append (2)\n      elif word in org_data:\n        seq.append (3)\n      else:\n        seq.append (0) \n    gazetteer_featureas.append(seq)\n  return (gazetteer_featureas)   \n"

In [37]:
#train_gazetteer_featureas = get_gazetteer_featureas (tr_tokenized_sents) 


In [38]:
#-------------- char embedding processing ------------------
char2idx = {"PAD": 0, "UNK": 1}
for c in "0123456789ABCDEFGHIJKLMNOPRSTUVWXYZabcdefghiklmnopqrstuvwxyíüءآأؤإئابةتثجحخدذرزسشصضطظعغفقكلمنهوىي–“”…ﻷﻹ":
  char2idx[c] = len(char2idx)
max_len_char = 20
def get_char_ids (sents):

  chars_ids = []
  for sent in sents:
    sent_char_seq = []
    for word in sent:
      word_char_seq = []       
      for char in word:
        if char in char2idx:
          word_char_seq.append (char2idx[char])
        else:
          word_char_seq.append (char2idx['UNK'])
      lw = max_len_char - len(word)
      word_char_seq += [0]*lw
      sent_char_seq.append(word_char_seq)
    l = max_len-len(sent)
    sent_char_seq += [[0]*max_len_char]*l
    chars_ids.append(sent_char_seq)
  return torch.tensor(chars_ids, dtype=torch.long)


In [39]:
train_chars = get_char_ids (tr_tokenized_sents)
val_chars = get_char_ids (val_tokenized_sents)
test_chars = get_char_ids (te_tokenized_sents)

In [40]:
print (te_tokenized_sents[0])
print (test_labels[0])
print (test_sents[0])

['[CLS]', 'ولم', 'يعد', 'من', 'الممكن', 'تبرير', 'سلوك', 'القوى', 'السياسية', 'الشيعية', 'للعرب', 'والمسلمين', 'لا', 'من', 'حيث', 'تحالف', 'هذه', 'القوى', 'مع', 'الاحتلال', 'وتصد', '##يها', 'للمقاومة', 'ولا', 'من', 'حيث', 'سياستها', 'وسلوك', '##ها', 'الطائفي', '[SEP]']
tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])
['ولم', 'يعد', 'من', 'الممكن', 'تبرير', 'سلوك', 'القوى', 'السياسية', 'الشيعية', 'للعرب', 'والمسلمين', 'لا', 'من', 'حيث', 'تحالف', 'هذه

In [41]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
train_batch_size = 32
test_batch_size = 8

# wrap tensors
#train_data = TensorDataset(train_ids, train_masks, train_seg, train_Fasttext_seq, train_labels)
train_data = TensorDataset(train_ids, train_masks, train_seg, train_labels)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, batch_size=train_batch_size, num_workers=2)

# wrap tensors
#val_data = TensorDataset(val_ids, val_masks, val_seg, val_Fasttext_seq, val_labels)
val_data = TensorDataset(val_ids, val_masks, val_seg, val_labels)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, batch_size=test_batch_size, num_workers=1)

# wrap tensors
#test_data = TensorDataset(test_ids, test_masks, test_seg, test_Fasttext_seq, test_labels)
test_data = TensorDataset(test_ids, test_masks, test_seg, test_labels)

# sampler for sampling the data during training
test_sampler = SequentialSampler(test_data)

# dataLoader for validation set
test_dataloader = DataLoader(test_data, batch_size=test_batch_size, num_workers=1)

In [42]:
from transformers import BertModel, BertConfig, AdamW
 
bert = BertModel.from_pretrained(bert_model)

In [43]:
class BERTNERTagger(nn.Module):
    def __init__(self,
                 bert):
        
        super().__init__()
        
        self.bert = bert 

        self.gruUnits = 384
      
      # dropout layer
        self.dropout = nn.Dropout(0.2)

        self.bidirectional = True
      # dense layer 1
        self.gru = nn.GRU(768, self.gruUnits,  bidirectional = self.bidirectional)
        
        self.fc = nn.Linear(self.gruUnits * 2 if self.bidirectional else self.gruUnits, len(tag2idx))
        #self.fc = nn.Linear(768, len(tag2idx))


      #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)


        self.attn = nn.MultiheadAttention(
                embed_dim=self.gruUnits * 2,
                num_heads=4,
                dropout=0.5
            )
        #self.embedding = nn.Embedding.from_pretrained(weights, padding_idx = 0)
        #self.embedding.weight.requires_grad = False
        
    def forward(self, input_id,masks, seg):

        pool,seq = self.bert(input_id, attention_mask=masks, token_type_ids=seg)

        pool = self.dropout (pool)

        #fast = self.embedding (Fasttext_seq)
        
        pool , h = self.gru(pool)
        pool = self.dropout (pool)

        #combined = self.dropout (combined)

      # output layer
        predictions = self.fc(pool)
        """
      
        log_likelihood = self.crf(logits, tags, mask)
        loss = -log_likelihood # Log likelihood is not normalized (It is not divided by the batch size).

        # To obtain the best sequence using viterbi decoding
        best_tag_sequence = self.crf.best_viterbi_tag(logits, mask)

        # To obtain output similar to the lstm prediction we can use the below code
        predictions = logits * 0.0
        for i, instance_tags in enumerate(best_tag_sequence):
            for j, tag_id in enumerate(instance_tags[0][0]):
                predictions[i, j, int(tag_id)] = 1
      # apply softmax activation
        #predictions = self.softmax(x)
        """

        return predictions

In [44]:
# pass the pre-trained BERT to our define architecture
model = BERTNERTagger(bert)

# push the model to GPU
model = model.to(device)

In [45]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
EPOCHS = 5
LEARNING_RATE = 5e-5
WARMUP_RATIO = 0.1
MAX_GRAD_NORM = 1.0
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.01,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

num_train_steps = int(len(train_sents) / train_batch_size * EPOCHS)
print('Number of training steps: ', num_train_steps)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#optimizer = AdamW(optimizer_parameters, lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(WARMUP_RATIO*num_train_steps), num_training_steps=num_train_steps
)


Number of training steps:  922


In [46]:
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss(ignore_index=0)
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss

In [47]:
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score


def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != 0:
                out_label_list[i].append(idx2tag[label_ids[i][j]])
                preds_list[i].append(idx2tag[preds[i][j]])
    return preds_list, out_label_list

def y2label(zipped, mask=0):
    out_true = []
    out_pred = []
    for zip_i in zipped:
        a, b = tuple(zip_i)
        if a != mask :
            out_true.append(idx2tag[a].replace('B-','').replace('I-',''))
            out_pred.append(idx2tag[b].replace('B-','').replace('I-',''))
    return out_true, out_pred

def compute_metrics(predictions,label_ids):
    print ("predictions:" , predictions.shape)
    print ("label_ids:" , label_ids.shape)
    predictions = np.argmax(predictions, axis=2)
    y_zipped = zip(np.array(label_ids).flat, np.array(predictions).flat)
    preds_list, out_label_list = y2label(y_zipped)
    print(classification_report(out_label_list, preds_list,digits=4))
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list, average='macro'),
        "recall": recall_score(out_label_list, preds_list, average='macro'),
        "f1": f1_score(out_label_list, preds_list, average='macro'),
    }

In [48]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    #sent_id, mask, seg, fast, labels = batch
    sent_id, mask, seg, labels = batch

    # clear previously calculated gradients 
    optimizer.zero_grad()        
    # get model predictions for the current batch
    preds = model(sent_id, mask, seg)
    # compute the loss between actual and predicted values
    # add on to the total loss
    loss = loss_fn(preds, labels, mask, len(tag2idx))
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()
    scheduler.step()
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [49]:
# function for evaluating the model
def evaluate(dataloader):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  total_labels = []
  # iterate over batches
  for step,batch in enumerate(dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, seg, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds= model(sent_id, mask, seg)
      # compute the validation loss between actual and predicted values
      loss = loss_fn(preds, labels, mask, len(tag2idx))
      total_loss = total_loss + loss  
      preds = preds.detach().cpu().numpy()
      labels=labels.detach().cpu().numpy()


      total_preds.append(preds)
      total_labels.append(labels)
  # compute the validation loss of the epoch
  avg_loss = total_loss / len(dataloader) 
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  total_labels  = np.concatenate(total_labels, axis=0)

  return avg_loss, total_preds, compute_metrics(total_preds,total_labels)

In [50]:
epochs = EPOCHS
MODEL_PATH = 'BERT_BGRU_softmax(ANERCorp_+_AQMAR).bin'
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
best_f1 = 0
#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
   
    #evaluate model
    valid_loss, preds, eval_metrics = evaluate(val_dataloader)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')
    print (eval_metrics)
    if eval_metrics['f1'] > best_f1:
      torch.save(model, MODEL_PATH)
      best_f1 = eval_metrics['f1']


 Epoch 1 / 5
  Batch    50  of    185.
  Batch   100  of    185.
  Batch   150  of    185.

Evaluating...
  Batch    50  of     84.
predictions: (665, 177, 10)
label_ids: (665, 177)
              precision    recall  f1-score   support

         LOC     0.9365    0.6987    0.8003       823
        MISC     0.6785    0.7846    0.7277       390
           O     0.9674    0.9909    0.9790     15743
         ORG     0.8375    0.5255    0.6458       510
        PERS     0.9533    0.9045    0.9283       880

    accuracy                         0.9563     18346
   macro avg     0.8746    0.7808    0.8162     18346
weighted avg     0.9556    0.9563    0.9540     18346


Training Loss: 0.363
Validation Loss: 0.166
{'accuracy_score': 0.9563392565136815, 'precision': 0.87462992102464, 'recall': 0.7808462119161483, 'f1': 0.8162108599595739}

 Epoch 2 / 5
  Batch    50  of    185.
  Batch   100  of    185.
  Batch   150  of    185.

Evaluating...
  Batch    50  of     84.
predictions: (665, 177, 

In [51]:
#model.load_state_dict(torch.load(MODEL_PATH))
test_loss, preds, test_metrics = evaluate (test_dataloader)
print(f"Test Loss = {test_loss}")
print(test_metrics)


Evaluating...
  Batch    50  of     92.
predictions: (729, 177, 10)
label_ids: (729, 177)
              precision    recall  f1-score   support

         LOC     0.9230    0.9256    0.9243       699
        MISC     0.8581    0.7741    0.8140       633
           O     0.9872    0.9917    0.9894     18060
         ORG     0.8640    0.8604    0.8622       480
        PERS     0.9676    0.9480    0.9577      1038

    accuracy                         0.9777     20910
   macro avg     0.9200    0.9000    0.9095     20910
weighted avg     0.9773    0.9777    0.9774     20910

Test Loss = 0.10893362760543823
{'accuracy_score': 0.9777140124342419, 'precision': 0.9199673475383315, 'recall': 0.8999575072111938, 'f1': 0.909507429220372}


In [52]:
test_sentence = "وقد حصل الفائزون على جوائز عينية من شركة سعادة للمواد الكهربائية"

In [53]:
test_sentence = test_sentence.split (" ")
print (test_sentence)

['وقد', 'حصل', 'الفائزون', 'على', 'جوائز', 'عينية', 'من', 'شركة', 'سعادة', 'للمواد', 'الكهربائية']


In [54]:
tokenized_test_sentence, _ = encode_sentence([test_sentence],[test_sentence])
print (tokenized_test_sentence)

[['[CLS]', 'وقد', 'حصل', 'الفائز', '##ون', 'على', 'جوائز', 'عينية', 'من', 'شركة', 'سعادة', 'للمواد', 'الكهربائية', '[SEP]']]


In [55]:
test_sentence_ids, test_sentence_masks, test_sentence_seg = sents_processing (tokenized_test_sentence)#


In [56]:
with torch.no_grad():
    output = model(test_sentence_ids.cuda(), test_sentence_masks.cuda(), test_sentence_seg.cuda())
output = output[0].detach().cpu().numpy()
tokens = tokenizer.convert_ids_to_tokens(test_sentence_ids.to('cpu').numpy()[0])
output = np.argmax(output, axis=1)
for i , tag in enumerate (output):
  if tokens[i] != '[PAD]' and tokens[i] != '[CLS]' and tokens[i] != '[SEP]':
    print (tokens[i], idx2tag[tag])

وقد O
حصل O
الفائز O
##ون O
على O
جوائز O
عينية O
من O
شركة O
سعادة B-ORG
للمواد O
الكهربائية O


In [57]:
print (tags)

NameError: ignored

In [ ]:
print (tag2idx)